In [1]:
45

45

In [6]:
import os
from dotenv import load_dotenv

import pandas as pd
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()

True

In [ ]:
GROQ_API_KEY =os.getenv("GROQ_API_KEY ")
os.getenv("GROQ_API_KEY ") = GROQ_API_KEY


In [7]:
import os
from langchain.chat_models import init_chat_model

model = init_chat_model("groq:llama-3.3-70b-versatile")
response = model.invoke("Why am i studying at DOT pune")
response

AIMessage(content="It seems like you're studying at the D Y Patil Institute of Technology (DY Patil) or possibly the Dr. D Y Patil Vidyapeeth (DYPU) in Pune, which has a department called the Department of Telecom (DOT). \n\nIf that is the case, you might be studying at DOT Pune for several reasons:\n\n1. **Reputation**: DY Patil is a well-known and reputable institution in Pune, offering high-quality education in various fields, including telecom engineering.\n2. **Course offerings**: The Department of Telecom at DOT Pune might offer a range of undergraduate and postgraduate courses in telecom engineering, such as B.Tech, M.Tech, or PhD programs, which align with your interests and career goals.\n3. **Infrastructure and facilities**: The institution might have state-of-the-art infrastructure, including modern labs, equipment, and facilities, which provide you with hands-on experience and exposure to the latest technologies in the field.\n4. **Faculty and research**: The faculty at DOT

DATA LOADING CLASS

In [8]:
class AnimeDataLoader:
    def __init__(self, original_csv: str, processed_csv: str):
        self.original_csv = original_csv
        self.processed_csv = processed_csv

    def load_and_process(self):
        df = pd.read_csv(
            self.original_csv,
            encoding="utf-8",
            on_bad_lines="skip"
        ).dropna()

        required_cols = {"Name", "Genres", "sypnopsis"}
        if not required_cols.issubset(df.columns):
            raise ValueError("Missing required columns in CSV")

        df["combined_info"] = (
            "Title: " + df["Name"]
            + " Overview: " + df["sypnopsis"]
            + " Genres: " + df["Genres"]
        )

        df[["combined_info"]].to_csv(
            self.processed_csv,
            index=False,
            encoding="utf-8"
        )

        return self.processed_csv


In [9]:
original_csv =  "data/anime_with_synopsis.csv"
processed_csv = "data/anime_updated.csv"

data_loader = AnimeDataLoader(original_csv, processed_csv)

data_loader.load_and_process()


'data/anime_updated.csv'

DATA SPLITTING


In [10]:
class VectorStoreBuilder:
    def __init__(self, csv_path: str, persist_dir: str = "chroma_db"):
        self.csv_path = csv_path
        self.persist_dir = persist_dir
        self.embedding = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2"
        )

    def build_and_save_vectorstore(self):
        loader = CSVLoader(
            file_path=self.csv_path,
            encoding="utf-8",
            metadata_columns=[]
        )

        documents = loader.load()

        splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=0
        )
        chunks = splitter.split_documents(documents)

        db = Chroma.from_documents(
            chunks,
            self.embedding,
            persist_directory=self.persist_dir
        )
        db.persist()

    def load_vector_store(self):
        return Chroma(
            persist_directory=self.persist_dir,
            embedding_function=self.embedding
        )


In [13]:
print("Building vector database...")

original_csv = "data/anime_with_synopsis.csv"
processed_csv = "data/anime_updated.csv"
persist_dir = "chroma_db"

loader = AnimeDataLoader(original_csv, processed_csv)
processed_csv_path = loader.load_and_process()

vector_builder = VectorStoreBuilder(
    processed_csv_path,
    persist_dir=persist_dir
)
vector_builder.build_and_save_vectorstore()

print("Vector database ready.")


Building vector database...
Vector database ready.


C:\Users\hp\AppData\Local\Temp\ipykernel_14244\3258009850.py:29: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [14]:
print(" STILL BUILDING A VECTOR STORE ")

vector_builder = VectorStoreBuilder(
    csv_path=processed_csv_path,
    persist_dir=persist_dir
)

vector_builder.build_and_save_vectorstore()

print("Vector database ready")

 STILL BUILDING A VECTOR STORE 
Vector database ready


RETRIVER TOOL

In [15]:
vector_builder = VectorStoreBuilder(
    csv_path="",
    persist_dir="chroma_db"
)
retriver = vector_builder.load_vector_store().as_retriever()

C:\Users\hp\AppData\Local\Temp\ipykernel_14244\3258009850.py:32: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  return Chroma(


In [16]:
query = "tell me about naruto"

retriver.invoke(query)

[Document(metadata={'row': 10, 'source': 'data/anime_updated.csv'}, page_content="combined_info: Title: Naruto Overview: oments prior to Naruto Uzumaki's birth, a huge demon known as the Kyuubi, the Nine-Tailed Fox, attacked Konohagakure, the Hidden Leaf Village, and wreaked havoc. In order to put an end to the Kyuubi's rampage, the leader of the village, the Fourth Hokage, sacrificed his life and sealed the monstrous beast inside the newborn Naruto. Now, Naruto is a hyperactive and knuckle-headed ninja still living in Konohagakure. Shunned because of the Kyuubi inside him, Naruto struggles to find his place in the village, while his burning desire to become the Hokage of Konohagakure leads him not only to some great new friends, but also some deadly foes. Genres: Action, Adventure, Comedy, Super Power, Martial Arts, Shounen"),
 Document(metadata={'source': 'data/anime_updated.csv', 'row': 10}, page_content="combined_info: Title: Naruto Overview: oments prior to Naruto Uzumaki's birth,

In [17]:
@tool
def anime_retriever_tool(query: str) -> str:
    """
    Use this tool to search the anime knowledge base.

    Always call this tool for anime-related questions such as:
    recommendations, similarity search, genres, or plot summaries.

    Input:
    - query: User's anime preference or question.

    Output:
    - Relevant anime information retrieved from the vector database.
    """
    docs = retriver.invoke(query)
    return "\n\n".join(doc.page_content for doc in docs)


In [18]:
model_with_tools = model.bind_tools([anime_retriever_tool])

In [19]:
messages = [
    {
        "role":"user",
        "content":"can you suggest something similar to naruto with a joke"
    }
]

In [20]:
ai_msg =model_with_tools.invoke(messages)
messages.append(ai_msg)


In [26]:
messages

[{'role': 'user', 'content': 'can you suggest something similar to naruto'},
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'hng4af11n', 'function': {'arguments': '{"query":"anime similar to Naruto"}', 'name': 'anime_retriever_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 335, 'total_tokens': 356, 'completion_time': 0.054485191, 'completion_tokens_details': None, 'prompt_time': 0.018389899, 'prompt_tokens_details': None, 'queue_time': 0.053136704, 'total_time': 0.07287509}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c1354-3f4e-7eb3-8895-ef72ca1f4adb-0', tool_calls=[{'name': 'anime_retriever_tool', 'args': {'query': 'anime similar to Naruto'}, 'id': 'hng4af11n', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 335, 'output_tok

In [21]:
for tool_call in ai_msg.tool_calls:
    tool_result = anime_retriever_tool.invoke(tool_call)
    messages.append(tool_result)

In [22]:
messages

[{'role': 'user',
  'content': 'can you suggest something similar to naruto with a joke'},
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '2v5brzv32', 'function': {'arguments': '{"query":"anime similar to Naruto with a joke"}', 'name': 'anime_retriever_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 338, 'total_tokens': 362, 'completion_time': 0.065051518, 'completion_tokens_details': None, 'prompt_time': 0.016822925, 'prompt_tokens_details': None, 'queue_time': 0.048707265, 'total_time': 0.081874443}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c6fd8-9b41-7c83-ac40-ca42a4bb1154-0', tool_calls=[{'name': 'anime_retriever_tool', 'args': {'query': 'anime similar to Naruto with a joke'}, 'id': '2v5brzv32', 'type': 'tool_call'}], invalid_tool_calls=[], usage_met

In [23]:
final_response = model_with_tools.invoke(messages)
print(final_response.text)